In [45]:
from pathlib import Path
from llama_index import download_loader, Document
from llama_index.text_splitter import SentenceSplitter
import csv

In [46]:
def read_csv(file_path: str) -> None:
    events_list = []
    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            events_list.append(row)
    return events_list

In [51]:
events

[{'Id': '1',
  'Name': 'The Phantom of the Opera',
  'One Line Description': 'Experience the iconic musical',
  'Location': 'Majestic Theatre NYC',
  'Time': 'September 5 2023 8:00 PM',
  'Avg. Rating': '4.6',
  'Ticket Price': '$75'},
 {'Id': '2',
  'Name': 'New York Knicks vs. Boston Celtics',
  'One Line Description': 'NBA game at Madison Square Garden',
  'Location': 'Madison Square Garden NYC',
  'Time': 'October 12 2023 7:30 PM',
  'Avg. Rating': '4.4',
  'Ticket Price': '$50'},
 {'Id': '3',
  'Name': 'Chicago Bulls vs. Los Angeles Lakers',
  'One Line Description': 'NBA showdown at United Center',
  'Location': 'United Center Chicago',
  'Time': 'November 3 2023 8:00 PM',
  'Avg. Rating': '4.5',
  'Ticket Price': '$60'},
 {'Id': '4',
  'Name': 'San Francisco Giants vs. Boston Red Sox',
  'One Line Description': 'MLB clash at Oracle Park',
  'Location': 'Oracle Park San Francisco',
  'Time': 'August 20 2023 1:10 PM',
  'Avg. Rating': '4.7',
  'Ticket Price': '$40'},
 {'Id': '5',


In [49]:
events = read_csv("./events.csv")
documents = [Document(text=str(t), metadata={
    "event_id": t["Id"], "event_name":  t["Name"] 
}) for t in events]